### Run BASiCS for estimation method comparisons

In [1]:
library(Seurat)
library(SeuratData)
library(cowplot)
library(patchwork)
library(scHOT)
library(tidyverse)
library(dplyr)
library(BiocParallel)
library(readr)
library(Matrix)

Attaching SeuratObject

Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element 

In [2]:
library(Seurat)
library(SeuratData)
library(SeuratDisk)

Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat



In [3]:
library(BASiCS)

Loading required package: SingleCellExperiment

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘matrixStats’


The following object is masked from ‘package:dplyr’:

    count



Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQR

### Convert h5ad to seurat objects for simulations

In [4]:
setwd('/data_volume/memento/simulation/inference/')

filenames <- c('de', 'dv')

for (f in filenames){
    filename = paste(
        f,
        '.h5ad',
        sep=''
    )
    Convert(filename, dest = "h5seurat", overwrite = TRUE, verbose = FALSE)
        
}

Warning message:
“Unknown file type: h5ad”
Warning message:
“'assay' not set, setting to 'RNA'”
Creating h5Seurat file for version 3.1.5.9900

Warning message:
“Unknown file type: h5ad”
Warning message:
“'assay' not set, setting to 'RNA'”
Creating h5Seurat file for version 3.1.5.9900



### BASiCS for simulations

In [5]:
runBASiCS <- function(seurat, group){
    Idents(seurat) <- 'ct_real'
    dat <- subset(x = seurat, idents = group)
    dat$batch <- 1
    dat.sce <- SingleCellExperiment(
    assays=list(counts = GetAssayData(dat)),
    colData = data.frame(BatchInfo = dat$batch))
    Chain <- BASiCS_MCMC(
      dat.sce,
      N = 20000, Thin = 20, Burn = 10000, WithSpikes = FALSE, SubsetBy = 'cell',
      PrintProgress = TRUE, Regression = TRUE,Threads = getOption("Ncpus", 14),)
    return(Chain)
}

In [ ]:
for (f in filenames){
    print(f)

    infilename = paste(
        f,
        '.h5seurat',
        sep=''
    )

    data_obj <- LoadH5Seurat(infilename, verbose = FALSE)

    A_chain <- runBASiCS(data_obj, 'A')
    B_chain <- runBASiCS(data_obj, 'B')
    
    saveRDS(A_chain, file = paste(f, "_A_20000.rds", sep=""))
    saveRDS(A_chain, file = paste(f, "_B_20000.rds", sep=""))

}


[1] "de"


Validating h5Seurat file

Warning message in .ChecksBASiCS_Data(Data, WithSpikes):
“If spike-in genes are not available, BASiCS recommends that the data contain at least 2 batches of cells (for the same population)
”


### run BASiCS for smFISH comparison

In [3]:
setwd('/data_volume/memento/saver/')
data_path <- '/data_volume/memento/saver/'

In [10]:
Convert('dropseq.h5ad', dest = "h5seurat", overwrite = TRUE, verbose = FALSE)

Warning message:
“Unknown file type: h5ad”
Warning message:
“'assay' not set, setting to 'RNA'”
Creating h5Seurat file for version 3.1.5.9900



In [12]:
runBASiCS_dropseq <- function(seurat, stat){
    Idents(seurat) <- 'group'
    dat <- seurat
    dat$batch <- 1
    dat.sce <- SingleCellExperiment(
    assays=list(counts = GetAssayData(dat)),
    colData = data.frame(BatchInfo = dat$batch))
    Chain <- BASiCS_MCMC(
      dat.sce,
      N = 1000, Thin = 10, Burn = 500, WithSpikes = FALSE, SubsetBy = 'cell',
      PrintProgress = FALSE, Regression = TRUE,Threads = getOption("Ncpus", 10),)
    df = as.data.frame(displayChainBASiCS(Chain, Param = stat))
    return(df)
}

In [ ]:
data_obj <- LoadH5Seurat('dropseq.h5seurat', verbose = FALSE)

mu <- runBASiCS_dropseq(data_obj, 'mu')
write_csv(mu, file = 'dropseq_BASiCS_mu.csv')

delta <- runBASiCS_dropseq(data_obj, 'delta')
write_csv(delta, file = 'dropseq_BASiCS_delta.csv')

Validating h5Seurat file

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message in .ChecksBASiCS_Data(Data, WithSpikes):
“If spike-in genes are not available, BASiCS recommends that the data contain at least 2 batches of cells (for the same population)
”
Running no spikes BASiCS sampler (regression case) ... 


